In [1]:
import pathlib
import pprint
import random

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
from orchid import (
    core as oc,
    net_quantity as onq,
    unit_system as units,
)

In [4]:
import option
import toolz.curried as toolz

In [5]:
# noinspection PyUnresolvedReferences
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import Array, Int32

In [6]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
    'permian': 'Project_frankNstein_Permian_UTM13_FEET.ifrac'
}

In [7]:
projects = toolz.pipe(
    project_filenames,
    toolz.valmap(lambda fn: str(pathlib.Path(r'c:\src\Orchid.IntegrationTestData').joinpath(fn))),
    toolz.valmap(toolz.curry(oc.load_project)),
)
pprint.pprint(projects)

{'bakken': <orchid.project.Project object at 0x000001FA1A4BB6D0>,
 'montney': <orchid.project.Project object at 0x000001FA1A4B0820>,
 'permian': <orchid.project.Project object at 0x000001FA1A4BDD90>}


In [8]:
native_projects = toolz.valmap(lambda p: p.dom_object, projects)
pprint.pprint(native_projects)

{'bakken': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x000001FA1A4BDC70>,
 'montney': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x000001FA1A4BDD60>,
 'permian': <Orchid.FractureDiagnostics.Factories.Implementations.Project object at 0x000001FA1A4BDEE0>}


In [9]:
wells = toolz.pipe(native_projects,
    toolz.valmap(lambda native_project: native_project.Wells.Items),
    toolz.valmap(lambda well_map: [(w.Name, w) for w in well_map]),
    toolz.valmap(dict))
pprint.pprint(wells)

{'bakken': {'Demo_1H': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8D00>,
            'Demo_2H': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8D30>,
            'Demo_3H': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8D60>,
            'Demo_4H': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8D90>},
 'montney': {'Hori_01': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8DC0>,
             'Hori_02': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8DF0>,
             'Hori_03': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8E20>,
             'Vert_01': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x000001FA440C8E50>},
 'permian': {'C1': <Orchid.FractureDiagnostics.Factories.Implementations.Well object at 0x

In [10]:
trajectories = toolz.valmap(toolz.valmap(lambda w: w.Trajectory), wells)
pprint.pprint(trajectories)

{'bakken': {'Demo_1H': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB100>,
            'Demo_2H': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB130>,
            'Demo_3H': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB160>,
            'Demo_4H': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB190>},
 'montney': {'Hori_01': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA1A4B0D60>,
             'Hori_02': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB1C0>,
             'Hori_03': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB220>,
             'Vert_01': <Orchid.FractureDiagnostics.Factories.Implementations.WellTrajectory object at 0x000001FA440CB250>},
 'permian'

In [11]:
project_units = toolz.valmap(lambda p: p.project_units, projects)
pprint.pprint(project_units)

{'bakken': <enum 'UsOilfield'>,
 'montney': <enum 'Metric'>,
 'permian': <enum 'UsOilfield'>}


In [12]:
project_name = 'bakken'
well_name = 'Demo_1H'
samples = [0, 252, 146, 174, 99, 185]

In [13]:
eastings = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetEastingArray(WellReferenceFrameXy.Project),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(eastings)

['-12994.1290 ft',
 '-23009.8780 ft',
 '-13280.2590 ft',
 '-15903.3980 ft',
 '-12731.8390 ft',
 '-16947.1980 ft']


In [14]:
northings = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetNorthingArray(WellReferenceFrameXy.Project),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(northings)

['35548.8500 ft',
 '36879.1400 ft',
 '36869.5100 ft',
 '36872.5900 ft',
 '36546.6200 ft',
 '36880.4600 ft']


In [15]:
tvdSss = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetTvdArray(DepthDatum.SeaLevel),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(tvdSss)

['0.0000 ft',
 '10705.2410 ft',
 '10769.8474 ft',
 '10750.7877 ft',
 '8609.4420 ft',
 '10739.3417 ft']


In [16]:
inclinations = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetInclinationArray(),
    toolz.map(lambda ne: onq.as_measurement(units.Common.ANGLE, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(inclinations)

['0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg']


In [17]:
azimuths = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetAzimuthEastOfNorthArray(),
    toolz.map(lambda ne: onq.as_measurement(units.Common.ANGLE, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(azimuths)

['0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg',
 '0.0000 deg']


In [18]:
mdKbs = toolz.pipe(
    trajectories,
    toolz.get_in([project_name, well_name]),
    lambda t: t.GetMdKbArray(),
    toolz.map(lambda ne: onq.as_measurement(project_units[project_name].LENGTH, option.maybe(ne))),
    list,
    toolz.get(samples),
    toolz.map(lambda e: f'{e.magnitude:.4f} {e.units:~P}'),
    list,
)
pprint.pprint(mdKbs)

['0.0000 ft',
 '21000.0000 ft',
 '11266.0000 ft',
 '13890.0000 ft',
 '8697.0000 ft',
 '14934.0000 ft']


In [19]:
frame_data = {
    'field': [project_name] * len(samples),
    'well': [well_name] * len(samples),
    'index': samples,
    'easting': eastings,
    'northing': northings,
    'tvdss': tvdSss,
    'inclination': inclinations,
    'azimuth': azimuths,
    'mdkb': mdKbs,
}

In [20]:
pd.DataFrame(data=frame_data)

,field,well,index,easting,northing,tvdss,inclination,azimuth,mdkb
0,bakken,Demo_1H,0,-12994.1290 ft,35548.8500 ft,0.0000 ft,0.0000 deg,0.0000 deg,0.0000 ft
1,bakken,Demo_1H,252,-23009.8780 ft,36879.1400 ft,10705.2410 ft,0.0000 deg,0.0000 deg,21000.0000 ft
2,bakken,Demo_1H,146,-13280.2590 ft,36869.5100 ft,10769.8474 ft,0.0000 deg,0.0000 deg,11266.0000 ft
3,bakken,Demo_1H,174,-15903.3980 ft,36872.5900 ft,10750.7877 ft,0.0000 deg,0.0000 deg,13890.0000 ft
4,bakken,Demo_1H,99,-12731.8390 ft,36546.6200 ft,8609.4420 ft,0.0000 deg,0.0000 deg,8697.0000 ft
5,bakken,Demo_1H,185,-16947.1980 ft,36880.4600 ft,10739.3417 ft,0.0000 deg,0.0000 deg,14934.0000 ft
